In [4]:
import imp
model = imp.load_source('model', '../modules/model.py')
svhn_images = imp.load_source('svhn_images', '../modules/svhn_images.py')

In [5]:
tf_graph = model.getGraph()
session = model.init_interactive_session(tf_graph)

In [6]:
model.load_saved_model('/flat2/saved_models/SVHN_3_digit_May7', session)

In [8]:
#import matplotlib.image as mpimg
#import matplotlib.pyplot as plt
#import matplotlib.patches as patches
#from matplotlib.patches import Rectangle
#from PIL import Image
#%matplotlib inline



Help on function download_and_extract_SVHN in module svhn_images:

download_and_extract_SVHN(data_kind, outpath, mat_file)
    Download and extract SVHN from its URL. data_kind should be either "test" or "train" 
    Function then produces a directory structure so each image has a text label file describing it



# Download SVHN and extract from label file

In [2]:
import os

outpath='/mnt/sandbox/raw'
if not os.path.exists(outpath): 
    os.mkdir(outpath)

for data_kind in ['train', 'test']:
    print "Processing %s" % data_kind

    svhn_images.download_and_extract_SVHN(data_kind, outpath, '../data/SVHN/%s' % data_kind)

    print "Finished processing %s" % data_kind

Processing train
Finished downloading train
Extracting train
Finished extracting train
Starting image 1.png 
Starting image 1001.png 
Starting image 2001.png 
Starting image 3001.png 
Starting image 4001.png 
Starting image 5001.png 
Starting image 6001.png 
Starting image 7001.png 
Starting image 8001.png 
Starting image 9001.png 
Starting image 10001.png 
Starting image 11001.png 
Starting image 12001.png 
Starting image 13001.png 
Starting image 14001.png 
Starting image 15001.png 
Starting image 16001.png 
Starting image 17001.png 
Starting image 18001.png 
Starting image 19001.png 
Starting image 20001.png 
Starting image 21001.png 
Starting image 22001.png 
Starting image 23001.png 
Starting image 24001.png 
Starting image 25001.png 
Starting image 26001.png 
Starting image 27001.png 
Starting image 28001.png 
Starting image 29001.png 
Starting image 30001.png 
Starting image 31001.png 
Starting image 32001.png 
Starting image 33001.png 
Completed 33402 images
Finished processing